In [48]:
import os
import re
import json
import time
from datetime import datetime
from typing import List, Dict, Any
import pandas as pd

# ノートブックでは初期値で__file__が設定されないため使用できない
# script_path = os.path.abspath(__file__)

CURRENT_DIR_PATH = os.getcwd()
SETTING_DIR_PATH = os.path.join(CURRENT_DIR_PATH,'setting')

In [49]:
CONFIG_PATH = os.path.join(SETTING_DIR_PATH, 'config.json')
with open(CONFIG_PATH, 'r', encoding="utf-8-sig") as config_file:
    CONFIG = json.load(config_file)


In [68]:
LOGGING_DATA_PATH = CONFIG["LOGGING_DATA_PATH"]
OUTPUT_FOLDER_PATH = CONFIG["OUTPUT_FOLDER_PATH"]
PREPROCESSED_FILE_PATH = os.path.join(OUTPUT_FOLDER_PATH, 'preprocessed_file_history_converted.json')


# スケール情報のJSONファイル
SCALE_JSON_PATH = os.path.join(SETTING_DIR_PATH,'wsn_scale.json')
SENS_TYPE_JSON_PATH = os.path.join(SETTING_DIR_PATH, 'sens_type.json')


# 最新値を保存してあるExcelシートの名称
CURRENT_SENSOR_READINGS_JSON = os.path.join(SETTING_DIR_PATH, 'current_sensor_readings.json')

# 管理台帳のパス
MANAGEMENT_LEDGER_PATH = CONFIG["MANAGEMENT_LEDGER_PATH"]
MANAGEMENT_LEDGER_SHEET_NAME = 'Sheet1'

In [51]:
def get_node_folders(logging_folder_path):
    # フォルダ内の全てのファイルとフォルダを取得
    all_entries = os.listdir(logging_folder_path)
    
    # "node"で始まるフォルダのみをフィルタリング
    node_folders = [
        os.path.join(logging_folder_path, entry) for entry in all_entries 
        if os.path.isdir(os.path.join(logging_folder_path, entry)) and entry.startswith("node")
    ]
    
    return node_folders

In [52]:
node_dir_list = get_node_folders(LOGGING_DATA_PATH)

In [53]:
def extract_node_ids(node_path):
    # 正規表現で "node" に続く数字の範囲を抽出
    match = re.search(r'node(\d+)-(\d+)', node_path)
    
    if match:
        start_id = int(match.group(1))
        end_id = int(match.group(2))
        return start_id, end_id
    else:
        raise ValueError("The path does not contain a valid 'node' range.")

In [54]:
for node_dir in node_dir_list:
    start_id, end_id = extract_node_ids(node_dir)
    print(f"Node_IDs: {start_id} to {end_id}")


Node_IDs: 1 to 17
Node_IDs: 1001 to 1017
Node_IDs: 1018 to 1034
Node_IDs: 1035 to 1051
Node_IDs: 1052 to 1068
Node_IDs: 18 to 34
Node_IDs: 3001 to 3017
Node_IDs: 3018 to 3034
Node_IDs: 3035 to 3051
Node_IDs: 35 to 51
Node_IDs: 52 to 68
Node_IDs: 6001 to 6017
Node_IDs: 69 to 85
Node_IDs: 86 to 100


In [55]:
today = datetime.today().strftime('%Y%m%d')

In [56]:
with open(SCALE_JSON_PATH, 'r', encoding='utf-8') as file:
    df_scale = pd.DataFrame(json.load(file))

In [57]:
df_scale

,scale_code_dec,scale_code_hex,scale
0,0,00,1.000
1,1,01,10.000
2,2,02,100.000
3,3,03,1000.000
4,4,04,0.100
5,5,05,0.010
6,6,06,0.001
7,128,80,-1.000
8,129,81,-10.000
9,130,82,-100.000


In [58]:
def load_sensor_ledger(path: str, sheet_name: str) -> pd.DataFrame:
    """
    センサ管理台帳を読み込む。
    """
    try:
        return pd.read_excel(path, sheet_name=sheet_name, engine='openpyxl')
    except Exception as e:
        raise
    
sensor_ledger = load_sensor_ledger(MANAGEMENT_LEDGER_PATH, MANAGEMENT_LEDGER_SHEET_NAME)

In [59]:
# センサ管理台帳
sensor_ledger.head()

,ID,測定対象,センサ種別,拠点,設置場所(棟),設置場所(階),設置場所(詳細),センサ製造番号,接続GW,MDASゲートウェイ(Port番号)
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,CT-105-5ファン,加速度版振動センサ,能美,A,屋外,CT-105,000710133433,kan1018gw-a01,GW1(55040)
2,3,CDP-105,加速度版振動センサ,能美,A,1,機械室(奥),00071013344b,kan1018gw-a01,GW1(55040)
3,4,CT-105-1/105-2,電流センサ(2ch/非防水),能美,A,1,機械室,00061013A92C,kan1018gw-a01,GW1(55040)
4,5,CT-105-3/105-4,電流センサ(2ch/非防水),能美,A,1,機械室,00061013A8CF,kan1018gw-a01,GW1(55040)


In [60]:
def load_sensor_sheets(json_path: str) -> List[Dict[str, Any]]:
    """
    センサーシート情報をJSONから読み込む。
    """
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            return json.load(f)
    except Exception as e:
        raise
    
sensor_sheets = load_sensor_sheets(CURRENT_SENSOR_READINGS_JSON)

In [21]:
sensor_sheets

[{'sheet_name': '温湿度センサ', 'dataframe': None},
 {'sheet_name': '熱電対センサ(2ch防水)', 'dataframe': None},
 {'sheet_name': '3本サーミスタ(3ch非防水)', 'dataframe': None},
 {'sheet_name': '加速度版振動センサ', 'dataframe': None},
 {'sheet_name': '4-20mAアナログ無線化ユニット(1ch非防水)', 'dataframe': None},
 {'sheet_name': '電流センサ(2ch非防水)', 'dataframe': None},
 {'sheet_name': '0-5Vアナログ無線化ユニット(1ch非防水)', 'dataframe': None},
 {'sheet_name': '4-20mAアナログ無線化ユニット(3ch防水)', 'dataframe': None},
 {'sheet_name': '0-5Vアナログ無線化ユニット(3ch防水)', 'dataframe': None},
 {'sheet_name': '接点パルス(3ch防水)', 'dataframe': None},
 {'sheet_name': '漏水センサ', 'dataframe': None},
 {'sheet_name': '振動センサType2TF-001(速度モード)', 'dataframe': None},
 {'sheet_name': '振動センサType2TF-001(低速回転モード)', 'dataframe': None},
 {'sheet_name': '振動センサType2TF-001(加速度モード)', 'dataframe': None},
 {'sheet_name': '防水中継器', 'dataframe': None}]

In [22]:
def clean_sheet_names(sensor_sheets: List[Dict[str, Any]]) -> None:
    """
    シート名に含まれる"/"を削除する。
    """
    for sheet in sensor_sheets:
        sheet['sheet_name'] = sheet['sheet_name'].replace('/', '')

In [23]:
clean_sheet_names(sensor_sheets)
sensor_sheets

[{'sheet_name': '温湿度センサ', 'dataframe': None},
 {'sheet_name': '熱電対センサ(2ch防水)', 'dataframe': None},
 {'sheet_name': '3本サーミスタ(3ch非防水)', 'dataframe': None},
 {'sheet_name': '加速度版振動センサ', 'dataframe': None},
 {'sheet_name': '4-20mAアナログ無線化ユニット(1ch非防水)', 'dataframe': None},
 {'sheet_name': '電流センサ(2ch非防水)', 'dataframe': None},
 {'sheet_name': '0-5Vアナログ無線化ユニット(1ch非防水)', 'dataframe': None},
 {'sheet_name': '4-20mAアナログ無線化ユニット(3ch防水)', 'dataframe': None},
 {'sheet_name': '0-5Vアナログ無線化ユニット(3ch防水)', 'dataframe': None},
 {'sheet_name': '接点パルス(3ch防水)', 'dataframe': None},
 {'sheet_name': '漏水センサ', 'dataframe': None},
 {'sheet_name': '振動センサType2TF-001(速度モード)', 'dataframe': None},
 {'sheet_name': '振動センサType2TF-001(低速回転モード)', 'dataframe': None},
 {'sheet_name': '振動センサType2TF-001(加速度モード)', 'dataframe': None},
 {'sheet_name': '防水中継器', 'dataframe': None}]

In [43]:
df_sens_type = pd.read_json(SENS_TYPE_JSON_PATH, encoding="utf-8")

In [44]:
df_sens_type.head()

,sens_code_dec,sens_code_hex,sens_type,sens_V,sens_data_type_1,sens_data_type_2,sens_data_type_3,sens_data_type_4,sens_data_type_5,sens_data_type_6,sens_data_type_7,sens_data_type_8,sens_data_type_9,sens_data_type_10,sens_data_type_11,sens_data_type_12,sens_data_type_13,sens_data_type_14
0,9.0,9.0,振動(加速度版）,電源電圧[V],加速度ピーク周波数1[Hz],ピーク加速度1[m/s2],加速度ピーク周波数2[Hz],ピーク加速度2[m/s2],加速度ピーク周波数3[Hz],ピーク加速度3[m/s2],加速度ピーク周波数4[Hz],ピーク加速度4[m/s2],加速度ピーク周波数5[Hz],ピーク加速度5[m/s2],加速度RMS[m/s2],尖り度[-],温度[℃],NaN
1,24.0,18.0,振動(速度版）,電源電圧[V],加速度ピーク周波数1[Hz],ピーク加速度1[m/s2],加速度ピーク周波数2[Hz],ピーク加速度2[m/s2],加速度RMS[m/s2],速度ピーク周波数1[Hz],ピーク速度1[mm/s],速度ピーク周波数2[Hz],ピーク速度2[mm/s],速度ピーク周波数3[Hz],ピーク速度3[m/s],速度RMS[mm/s],尖り度[-],温度[℃]
2,18.0,12.0,CTセンサ(2ch/非防水),電源電圧[V],パケット番号[times],電流値1[A],電流値2[A],借電状態[-],前回電流値1[A],前回電流値2[A],前回借電状態[-],NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,1.0,温湿度,電源電圧[V],温度[℃],湿度[%RH],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7.0,7.0,熱電対,電源電圧[V],温度1[℃],温度2[℃],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
node_folders = get_node_folders(LOGGING_DATA_PATH)

In [25]:
node_folders

['\\\\m5fsv01\\KMM共有領域\\KA5300\\J_環境管理\\80_施設MDASプロジェクト\\KA1018_nomi\\A\\LoggingLog\\node1-17',
 '\\\\m5fsv01\\KMM共有領域\\KA5300\\J_環境管理\\80_施設MDASプロジェクト\\KA1018_nomi\\A\\LoggingLog\\node1001-1017',
 '\\\\m5fsv01\\KMM共有領域\\KA5300\\J_環境管理\\80_施設MDASプロジェクト\\KA1018_nomi\\A\\LoggingLog\\node1018-1034',
 '\\\\m5fsv01\\KMM共有領域\\KA5300\\J_環境管理\\80_施設MDASプロジェクト\\KA1018_nomi\\A\\LoggingLog\\node1035-1051',
 '\\\\m5fsv01\\KMM共有領域\\KA5300\\J_環境管理\\80_施設MDASプロジェクト\\KA1018_nomi\\A\\LoggingLog\\node1052-1068',
 '\\\\m5fsv01\\KMM共有領域\\KA5300\\J_環境管理\\80_施設MDASプロジェクト\\KA1018_nomi\\A\\LoggingLog\\node18-34',
 '\\\\m5fsv01\\KMM共有領域\\KA5300\\J_環境管理\\80_施設MDASプロジェクト\\KA1018_nomi\\A\\LoggingLog\\node3001-3017',
 '\\\\m5fsv01\\KMM共有領域\\KA5300\\J_環境管理\\80_施設MDASプロジェクト\\KA1018_nomi\\A\\LoggingLog\\node3018-3034',
 '\\\\m5fsv01\\KMM共有領域\\KA5300\\J_環境管理\\80_施設MDASプロジェクト\\KA1018_nomi\\A\\LoggingLog\\node3035-3051',
 '\\\\m5fsv01\\KMM共有領域\\KA5300\\J_環境管理\\80_施設MDASプロジェクト\\KA1018_nomi\\A\\LoggingLog\\node35-51',
 '\

In [61]:
def is_file_processed(file_path: str, json_file_path: str = PREPROCESSED_FILE_PATH) -> bool:
    """
    ファイルが既に処理済みかどうかを判定する。
    """
    if not os.path.exists(json_file_path):
        return False
    try:
        with open(json_file_path, 'r') as json_file:
            data = json.load(json_file)
            if "preprocessed_file_path" not in data:
                return False
            for entry in data["preprocessed_file_path"]:
                if entry.get("file_path") == file_path:
                    return True
    except (json.JSONDecodeError, IOError) as e:
        print(f"Error reading JSON file: {e}")
    return False

def is_files_processed(node_folder: str, json_file_path: str = PREPROCESSED_FILE_PATH) -> bool:
    """
    フォルダ内の全てのファイルが処理済みかどうかを判定する。
    """
    for root, _, files in os.walk(node_folder):
        for file in files:
            file_path = os.path.join(root, file)
            if not is_file_processed(file_path, json_file_path):
                return False
    return True


In [ ]:
def get_unprocessed_files(node_folders: list, json_file_path: str) -> list:
    """
    指定フォルダ群内の未処理ファイルのパス一覧を返す。
    Args:
        node_folders (list): 対象フォルダのパスリスト
        json_file_path (str): 処理済みファイル履歴(json)
    Returns:
        list: 未処理ファイルの絶対パスリスト
    """
    # 処理済みファイルパスのセットを作成
    processed_files = set()
    if os.path.exists(json_file_path):
        with open(json_file_path, 'r', encoding='utf-8') as f:
            try:
                data = json.load(f)
                for entry in data.get("preprocessed_file_path", []):
                    processed_files.add(entry.get("file_path"))
                print(len(processed_files))
            except Exception:
                print("Error reading JSON file.")
                pass
          
    # 未処理ファイルのパスを収集
    unprocessed_files = []
    for folder in node_folders:
        for file in os.listdir(folder):
            file_path = os.path.join(folder, file)
            if os.path.isfile(file_path) and file_path not in processed_files:
                unprocessed_files.append(file_path)
    return unprocessed_files

In [70]:
unprocessed_files = get_unprocessed_files(node_dir_list, PREPROCESSED_FILE_PATH)
unprocessed_files

start
if OK
817


['\\\\m5fsv01\\KMM共有領域\\KA5300\\J_環境管理\\80_施設MDASプロジェクト\\KA1018_nomi\\A\\LoggingLog\\node1-17\\node1-17_20241002.CSV',
 '\\\\m5fsv01\\KMM共有領域\\KA5300\\J_環境管理\\80_施設MDASプロジェクト\\KA1018_nomi\\A\\LoggingLog\\node1-17\\node1-17_20241003.CSV',
 '\\\\m5fsv01\\KMM共有領域\\KA5300\\J_環境管理\\80_施設MDASプロジェクト\\KA1018_nomi\\A\\LoggingLog\\node1-17\\node1-17_20241004.CSV',
 '\\\\m5fsv01\\KMM共有領域\\KA5300\\J_環境管理\\80_施設MDASプロジェクト\\KA1018_nomi\\A\\LoggingLog\\node1-17\\node1-17_20241005.CSV',
 '\\\\m5fsv01\\KMM共有領域\\KA5300\\J_環境管理\\80_施設MDASプロジェクト\\KA1018_nomi\\A\\LoggingLog\\node1-17\\node1-17_20241006.CSV',
 '\\\\m5fsv01\\KMM共有領域\\KA5300\\J_環境管理\\80_施設MDASプロジェクト\\KA1018_nomi\\A\\LoggingLog\\node1-17\\node1-17_20241007.CSV',
 '\\\\m5fsv01\\KMM共有領域\\KA5300\\J_環境管理\\80_施設MDASプロジェクト\\KA1018_nomi\\A\\LoggingLog\\node1-17\\node1-17_20241008.CSV',
 '\\\\m5fsv01\\KMM共有領域\\KA5300\\J_環境管理\\80_施設MDASプロジェクト\\KA1018_nomi\\A\\LoggingLog\\node1-17\\node1-17_20241009.CSV',
 '\\\\m5fsv01\\KMM共有領域\\KA5300\\J_環境管理\\80_施設MDA

In [ ]:

for node_folder in node_folders:
    start_node, end_node = extract_node_ids(node_folder)
    file_list = os.listdir(node_folder)
    print(start_node,end_node, "フォルダ内のファイル数：",len(file_list),"個")
    
    for preprocessing_file in file_list:
        file_path = os.path.join(node_folder, preprocessing_file)
        print(file_path)
    

1 17 フォルダ内のファイル数： 283 個
\\m5fsv01\KMM共有領域\KA5300\J_環境管理\80_施設MDASプロジェクト\KA1018_nomi\A\LoggingLog\node1-17\node1-17_20240716.CSV
\\m5fsv01\KMM共有領域\KA5300\J_環境管理\80_施設MDASプロジェクト\KA1018_nomi\A\LoggingLog\node1-17\node1-17_20240717.CSV
\\m5fsv01\KMM共有領域\KA5300\J_環境管理\80_施設MDASプロジェクト\KA1018_nomi\A\LoggingLog\node1-17\node1-17_20240718.CSV
\\m5fsv01\KMM共有領域\KA5300\J_環境管理\80_施設MDASプロジェクト\KA1018_nomi\A\LoggingLog\node1-17\node1-17_20240719.CSV
\\m5fsv01\KMM共有領域\KA5300\J_環境管理\80_施設MDASプロジェクト\KA1018_nomi\A\LoggingLog\node1-17\node1-17_20240720.CSV
\\m5fsv01\KMM共有領域\KA5300\J_環境管理\80_施設MDASプロジェクト\KA1018_nomi\A\LoggingLog\node1-17\node1-17_20240721.CSV
\\m5fsv01\KMM共有領域\KA5300\J_環境管理\80_施設MDASプロジェクト\KA1018_nomi\A\LoggingLog\node1-17\node1-17_20240722.CSV
\\m5fsv01\KMM共有領域\KA5300\J_環境管理\80_施設MDASプロジェクト\KA1018_nomi\A\LoggingLog\node1-17\node1-17_20240723.CSV
\\m5fsv01\KMM共有領域\KA5300\J_環境管理\80_施設MDASプロジェクト\KA1018_nomi\A\LoggingLog\node1-17\node1-17_20240724.CSV
\\m5fsv01\KMM共有領域\KA5300\J_環境管理\80_施設MDA